In [0]:
import pandas as pd
import numpy as np
import json


# Start Added for Fixed 5 decimal places - Amol G
def remove_nulls(d):
  return {k: v for k, v in d.items() if v is not None}

appliedTraitsDecimal = '[{"traitReference": "is.dataFormat.numeric.shaped","arguments": [{"name": "precision","value": "19"},{"name": "scale","value": "5"}]}]'
appliedTraitsPlaceHolder = '#appliedTraitsDecimal#'
# End

df = pd.read_csv("EngineBAuditCDMv1.9.csv")

df = df.rename(columns={
    "enginebAttribute": "name",
    "MS_DataType": "dataType",
    "primaryKey": "isNullable",
    "Description": "description"
})
df["isNullable"] = np.where(df["isNullable"].isnull(), "true", "false")
df["displayName"] = df["name"]
df["decimalPlaces"] = np.where(df["dataType"] == "decimal", 5, None)


# Start Added for Fixed 5 decimal places - Amol G

df["appliedTraits"] = np.where(df["dataType"] == "decimal",appliedTraitsPlaceHolder,None)

# End

df_grouped = df.groupby("enginebEntity")

for name, group in df_grouped:
    json_object = {
        "jsonSchemaSemanticVersion": "1.0.0",
        "imports": [
            {
                "corpusPath": "_allImports.cdm.json"
            }
        ],
        "definitions": [
            {
                "entityName": name,
                "extendsEntity": "CdmEntity",
                "hasAttributes": [
                    json.loads(group[["name", "dataType", "isNullable", "displayName", "description","appliedTraits"]].to_json(orient="records"),object_hook=remove_nulls)
                ]
            }
        ]
    }
    
    # Start Added for Fixed 5 decimal places - Amol G
    json_object = json.loads(json.dumps(json_object).replace('"#appliedTraitsDecimal#"',appliedTraitsDecimal))
    # End
    
    with open(f"output/{name}.json", "w") as outfile:
        json.dump(json_object, outfile, indent=4)